# Air Quality Barcelona

In [1]:
import pandas as pd

In [2]:
def structure_dataset(csv_name):
    df = pd.read_csv('data/'+csv_name)
    df.drop(['CODI_PROVINCIA','PROVINCIA','CODI_MUNICIPI','MUNICIPI'], axis = 1, inplace = True)
    df.rename(columns = {'CODI_CONTAMINANT':'POLLUTANT_CODE','ESTACIO':'STATION','ANY':'YEAR','MES':'MONTH','DIA':'DAY'}, inplace = True)
    for num in ["%.2d" % i for i in range(1,25)]:
        df.drop(['V'+num], axis = 1, inplace = True)
        df['H'+num] = pd.to_numeric(df['H'+num],errors = 'coerce')
    new_df = pd.DataFrame({'STATION':[df['STATION'][int(i/24)] for i in range(df.shape[0]*24)],
                      'POLLUTANT_CODE':[df['POLLUTANT_CODE'][int(i/24)] for i in range(df.shape[0]*24)],
                      'YEAR':[df['YEAR'][int(i/24)] for i in range(df.shape[0]*24)],
                      'MONTH':[df['MONTH'][int(i/24)] for i in range(df.shape[0]*24)],
                      'DAY':[df['DAY'][int(i/24)] for i in range(df.shape[0]*24)],
                      'HOUR':[j for i in range(df.shape[0]) for j in range(1,25)],
                      'VALUE':[df['H'+num][row] for row in range(df.shape[0]) for num in ["%.2d" % i for i in range(1,25)]]
                      })
    return new_df

In [3]:
jan2020 = structure_dataset('2020_01_Gener_qualitat_aire_BCN.csv')
feb2020 = structure_dataset('2020_02_Febrer_qualitat_aire_BCN.csv')
march2020 = structure_dataset('2020_03_Marc_qualitat_aire_BCN.csv')
april2020 = structure_dataset('2020_04_Abril_qualitat_aire_BCN.csv')
may2020 = structure_dataset('2020_05_Maig_qualitat_aire_BCN.csv')
june2020 = structure_dataset('2020_06_Juny_qualitat_aire_BCN.csv')
july2020 = structure_dataset('2020_07_Juliol_qualitat_aire_BCN.csv')
aug2020 = structure_dataset('2020_08_Agost_qualitat_aire_BCN.csv')
sep2020 = structure_dataset('2020_09_Setembre_qualitat_aire_BCN.csv')

In [40]:
stations = pd.read_csv('data/Qualitat_Aire_Estacions.csv')
pollutants = pd.read_csv('data/Qualitat_Aire_Contaminants.csv')

In [41]:
jan_sep_2020 = pd.concat([jan2020,feb2020,march2020,april2020,may2020,june2020,july2020,aug2020,sep2020],ignore_index = True)

In [42]:
stations.drop(['Codi_Contaminant'], axis = 1, inplace = True)
stations.drop_duplicates(inplace = True)
jan_sep_2020 = jan_sep_2020.merge(stations[['Estacio','Nom_districte']], how = 'left',left_on = 'STATION',right_on = 'Estacio')
jan_sep_2020.drop(['Estacio'], axis = 1, inplace = True)
jan_sep_2020.rename(columns = {'Nom_districte':'DISTRICT_NAME'}, inplace = True)

In [43]:
jan_sep_2020 = jan_sep_2020.merge(pollutants, how = 'left',left_on = 'POLLUTANT_CODE',right_on = 'Codi_Contaminant')
jan_sep_2020.drop(['Codi_Contaminant'], axis = 1, inplace = True)
jan_sep_2020.rename(columns = {'Desc_Contaminant':'POLLUTANT_DESCRIPTION','Unitats':'UNITS'}, inplace = True)

In [44]:
jan_sep_2020 = jan_sep_2020[['YEAR','MONTH','DAY','HOUR','STATION','DISTRICT_NAME','POLLUTANT_CODE','POLLUTANT_DESCRIPTION','UNITS','VALUE']]

## Air Quality by neighbourhood

In [68]:
jan_sep_2020.groupby(['POLLUTANT_DESCRIPTION','DISTRICT_NAME']).mean()['VALUE']

POLLUTANT_DESCRIPTION  DISTRICT_NAME 
CO                     Eixample           0.259083
                       Gracia             0.318769
                       Horta-Guinardo     0.276345
                       Les Corts          0.284462
NO                     Ciutat Vella       7.949243
                       Eixample          17.275962
                       Gracia            11.305618
                       Horta-Guinardo     7.169296
                       Les Corts          5.004723
                       Sant Marti         9.522316
                       Sants-Montjuic     6.026637
NO2                    Ciutat Vella      22.168063
                       Eixample          33.317340
                       Gracia            30.399679
                       Horta-Guinardo    21.205815
                       Les Corts         17.066792
                       Sant Marti        26.692755
                       Sants-Montjuic    21.951220
NOx                    Ciutat Vella      33.

## Air Quality by day

In [46]:
date_df = pd.DataFrame({'DATE':[str(jan_sep_2020['DAY'][i])+'/'+str(jan_sep_2020['MONTH'][i])+'/'+str(jan_sep_2020['YEAR'][i]) for i in range(jan_sep_2020.shape[0])]})

In [47]:
date_jan_sep_2020 = pd.concat([jan_sep_2020,date_df],axis=1)

In [49]:
date_jan_sep_2020['DATE'] = pd.to_datetime(date_jan_sep_2020['DATE'])

In [50]:
date_jan_sep_2020

,YEAR,MONTH,DAY,HOUR,STATION,DISTRICT_NAME,POLLUTANT_CODE,POLLUTANT_DESCRIPTION,UNITS,VALUE,DATE
0,2020,1,1,1,4,Sant Marti,7,NO,µg/m³,3.0,2020-01-01
1,2020,1,1,2,4,Sant Marti,7,NO,µg/m³,3.0,2020-01-01
2,2020,1,1,3,4,Sant Marti,7,NO,µg/m³,2.0,2020-01-01
3,2020,1,1,4,4,Sant Marti,7,NO,µg/m³,1.0,2020-01-01
4,2020,1,1,5,4,Sant Marti,7,NO,µg/m³,2.0,2020-01-01
...,...,...,...,...,...,...,...,...,...,...,...
304963,2020,9,30,20,58,NaN,14,O3,µg/m³,94.0,2020-09-30
304964,2020,9,30,21,58,NaN,14,O3,µg/m³,99.0,2020-09-30
304965,2020,9,30,22,58,NaN,14,O3,µg/m³,107.0,2020-09-30
304966,2020,9,30,23,58,NaN,14,O3,µg/m³,91.0,2020-09-30


In [56]:
date_jan_sep_2020['DAY_WEEK'] = [date_jan_sep_2020['DATE'][i].weekday() for i in range(date_jan_sep_2020.shape[0])]

In [58]:
date_jan_sep_2020.drop(['YEAR','MONTH','DAY'],axis=1,inplace=True)

In [59]:
date_jan_sep_2020

,HOUR,STATION,DISTRICT_NAME,POLLUTANT_CODE,POLLUTANT_DESCRIPTION,UNITS,VALUE,DATE,DAY_WEEK
0,1,4,Sant Marti,7,NO,µg/m³,3.0,2020-01-01,2
1,2,4,Sant Marti,7,NO,µg/m³,3.0,2020-01-01,2
2,3,4,Sant Marti,7,NO,µg/m³,2.0,2020-01-01,2
3,4,4,Sant Marti,7,NO,µg/m³,1.0,2020-01-01,2
4,5,4,Sant Marti,7,NO,µg/m³,2.0,2020-01-01,2
...,...,...,...,...,...,...,...,...,...
304963,20,58,NaN,14,O3,µg/m³,94.0,2020-09-30,2
304964,21,58,NaN,14,O3,µg/m³,99.0,2020-09-30,2
304965,22,58,NaN,14,O3,µg/m³,107.0,2020-09-30,2
304966,23,58,NaN,14,O3,µg/m³,91.0,2020-09-30,2


In [67]:
date_jan_sep_2020.groupby(['POLLUTANT_DESCRIPTION','DAY_WEEK']).mean()

HOUR    STATION  POLLUTANT_CODE      VALUE
POLLUTANT_DESCRIPTION DAY_WEEK                                            
CO                    0         12.5  49.500000             6.0   0.273906
                      1         12.5  49.500000             6.0   0.290930
                      2         12.5  49.500000             6.0   0.302486
                      3         12.5  49.500000             6.0   0.300582
                      4         12.5  49.500000             6.0   0.307892
                      5         12.5  49.500000             6.0   0.259771
                      6         12.5  49.500000             6.0   0.258017
NO                    0         12.5  44.000000             7.0   7.470194
                      1         12.5  44.000000             7.0   8.217404
                      2         12.5  44.000000             7.0   9.321800
                      3         12.5  44.000000             7.0  10.758750
                      4         12.5  44.000000             7.0  10.467450
                      5         12.5  44.000000             7.0   5.776205
                      6         12.5  44.000000             7.0   5.724605
NO2                   0         12.5  44.000000             8.0  21.992189
                      1         12.5  44.000000             8.0  23.378157
                      2         12.5  44.000000             8.0  24.143381
                      3         12.5  44.000000             8.0  26.392222
                      4         12.5  44.000000             8.0  25.648759
                      5         12.5  44.000000             8.0  18.767533
                      6         12.5  44.000000             8.0  18.227019
NOx                   0         12.5  44.000000            12.0  32.775772
                      1         12.5  44.000000            12.0  35.438312
                      2         12.5  44.000000            12.0  37.811318
                      3         12.5  44.000000            12.0  42.310139
                      4         12.5  44.000000            12.0  41.133826
                      5         12.5  44.000000            12.0  26.985542
                      6         12.5  44.000000            12.0  26.305029
O3                    0         12.5  51.000000            14.0  57.561527
                      1         12.5  51.000000            14.0  57.506813
                      2         12.5  51.000000            14.0  57.159776
                      3         12.5  51.000000            14.0  54.022324
                      4         12.5  51.000000            14.0  56.002853
                      5         12.5  51.000000            14.0  59.793055
                      6         12.5  51.000000            14.0  58.186286
PM10                  0         12.5  43.333333            10.0  18.353280
                      1         12.5  43.333333            10.0  19.204026
                      2         12.5  43.333333            10.0  20.149073
                      3         12.5  43.333333            10.0  20.817497
                      4         12.5  43.333333            10.0  22.387297
                      5         12.5  43.333333            10.0  19.421516
                      6         12.5  43.333333            10.0  16.897318
SO2                   0         12.5  49.500000             1.0   1.486531
                      1         12.5  49.500000             1.0   1.429557
                      2         12.5  49.500000             1.0   1.422877
                      3         12.5  49.500000             1.0   1.484865
                      4         12.5  49.500000             1.0   1.453993
                      5         12.5  49.500000             1.0   1.359833
                      6         12.5  49.500000             1.0   1.364173